本ノートブックでは、[Gemini Developer API](https://ai.google.dev/)を用いて、第7章のReActエージェントを構築するコードを紹介します。

In [14]:
!pip install -q google-genai==1.18.0 langchain==0.3.25 langchain-google-genai==2.1.5 langchain-community==0.3.24 duckduckgo-search==8.0.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.7 MB/s eta 0:00:00


In [ ]:
import os

from langchain.agents import load_tools, Tool, AgentExecutor, create_react_agent
from langchain.tools import DuckDuckGoSearchResults
from langchain_core.prompts.prompt import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

以下のページを参考に、APIキーを取得しましょう。

- [Gemini API キーを取得する](https://ai.google.dev/gemini-api/docs/api-key?hl=ja)

APIにはレート制限があります。詳細については、以下のページを参照してください。

- [レート制限](https://ai.google.dev/gemini-api/docs/rate-limits?hl=ja)

In [2]:
os.environ["GOOGLE_API_KEY"] = "APIキーを貼り付け"

次に、LangChainからGeminiを利用するために、`ChatGoogleGenerativeAI`をインスタンス化します。

In [7]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
)

動作確認をしておきます。以下では、英語を日本語に翻訳する指示をしています。

In [9]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to Japanese. Translate the user sentence.",
    ),
    (
        "human",
        "I love programming."
    ),
]
llm.invoke(messages)

AIMessage(content='プログラミングが大好きです。', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--967a964e-1426-4947-9a78-2c5e6a023783-0', usage_metadata={'input_tokens': 20, 'output_tokens': 7, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}})

以降は、Geminiのモデルを使う点以外は本書の説明と同じです。

In [10]:
react_template = """以下の質問に対してできる限り良い回答をしてください。次のツールにアクセスできます：

{tools}

以下の形式を使用してください：

Question: あなたが回答する必要のある質問
Thought: 何をすべきか常に考える
Action: 取るべき行動。以下のいずれかから選択します [{tool_names}]
Action Input: 行動に対する入力
Observation: 行動の結果
...(この思考/行動/行動の入力/観察結果はN回繰り返すことができます)
Thought: 最終回答がわかりました
Final Answer: 元の質問に対する最終回答

開始してください！

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [15]:
search = DuckDuckGoSearchResults()
search_tool = Tool(
  name="search",
  description="Web検索エンジン。一般的な質問に対する検索エンジンとして使用する。",
  func=search.run,
)
tools = load_tools(["llm-math"], llm=llm)
tools.append(search_tool)

In [16]:
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
agent=agent, tools=tools, verbose=True, handle_parsing_errors=True )

In [17]:
agent_executor.invoke(
    {"input": "MacBook Proの現在の価格はUSDでいくらですか? また、為替レートが1USD = 150JPYの場合、JPYではいくらになりますか?"}
)



> Entering new AgentExecutor chain...
MacBook Proの現在の価格をUSDで調べ、USDからJPYへの換算を行う必要がある。
Action: search
Action Input: "MacBook Pro price USD"snippet: Best MacBook Pro deals for June 2025. Save up to $500 on Apple 14-inch & 16-inch laptops with deals on every M4 model & clearance prices on M3 specs., title: Best MacBook Pro Deals for June 2025 | Save up to $500 - AppleInsider, link: https://appleinsider.com/deals/best-macbook-pro-deals, snippet: Retail MacBook Pro 14-inch prices start at $1,999. But check in frequently with AppleInsider to find the best 14-inch MacBook Pro deals that deliver the lowest and cheapest prices on every configuration. Configurations Discount Price Alert; M2 Pro (10-core CPU, 16-core GPU), 16GB, 512GB, Space Gray: $1,999:, title: MacBook Pro 14-inch M2 Pro & M2 Max Prices - AppleInsider, link: https://prices.appleinsider.com/macbook-pro-14-inch-2023, snippet: According to an article from CNET, the base price for the 14-inch M2 MacBook Pro starts at approximately

{'input': 'MacBook Proの現在の価格はUSDでいくらですか? また、為替レートが1USD = 150JPYの場合、JPYではいくらになりますか?',
 'output': 'MacBook Proの価格はモデルによって異なり、14インチモデルは1,999ドルから、16インチモデルは2,499ドルからとなっています。為替レートが1USD = 150JPYの場合、14インチモデルは約299,850円、16インチモデルは約374,850円になります。'}

## 参考資料

- [ChatGoogleGenerativeAI](https://python.langchain.com/docs/integrations/chat/google_generative_ai/)
- [Gemini API キーを取得する](https://ai.google.dev/gemini-api/docs/api-key?hl=ja)
- [Google Gen AI SDK](https://github.com/googleapis/python-genai)
- [レート制限](https://ai.google.dev/gemini-api/docs/rate-limits?hl=ja)